In [ ]:
# !pip install -q -U langchain openai chromadb tiktoken

In [1]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [22]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load blog post
# "https://blog.langchain.dev/langchain-prompt-hub/"
# https://blog.csdn.net/github_38592071/article/details/131733697
web_url = "https://blog.csdn.net/github_38592071/article/details/131733697"
loader = WebBaseLoader(web_url)
data = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)

In [13]:
splits

[Document(page_content='Announcing LangChain Hub\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBy LangChain\n\n\n\n\nRelease Notes\n\n\n\n\nCase Studies\n\n\n\n\nLangChain\n\n\n\n\nGitHub\n\n\n\n\nDocs\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAnnouncing LangChain Hub\n\nBy LangChain\n6 min read\nSep 5, 2023', metadata={'source': 'https://blog.langchain.dev/langchain-prompt-hub/', 'title': 'Announcing LangChain Hub', 'language': 'en'}),
 Document(page_content="Today, we're excited to launch LangChain Hub‚Äìa home for uploading, browsing, pulling, and managing your prompts. (Soon, we'll be adding other artifacts like chains and agents).\uf8ffüí°Explore the Hub hereLangChain Hub is built into LangSmith (more on that below) so there are 2 ways to start exploring LangChain Hub.With LangSmith access: Full read and write permissions. You can explore all existin

In [27]:
# CloudflareWorkersAI
from dotenv import load_dotenv
import os
from langchain_community.llms.cloudflare_workersai import CloudflareWorkersAI

# 加载当前目录下的.env文件
# load_dotenv()
# load_dotenv(override=True) 会重新读取.env
load_dotenv(override=True)

# 现在可以像访问普通环境变量一样访问.env文件中的变量了
account_id = os.getenv('CF_ACCOUNT_ID')
api_token = os.getenv('CF_API_TOKEN')

print(account_id)
print(api_token)

import getpass

model = '@cf/meta/llama-3-8b-instruct'
cf_llm = CloudflareWorkersAI(account_id=account_id, api_token=api_token, model=model)
# 最新的Embedding方式
# cloudflare_workersai
from langchain_community.embeddings.cloudflare_workersai import (
    CloudflareWorkersAIEmbeddings,
)

# //维度是：384
embedding = CloudflareWorkersAIEmbeddings(
    account_id=account_id,
    api_token=api_token,
    model_name="@cf/baai/bge-small-en-v1.5",
)

8483c3ec7a0cbc54a8d660b5b9002b04
Gcllof8ze6dgtcqFI5FQZ2SD_5tfCD4Db7NuS6jn


In [23]:
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

In [19]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

# 加载当前目录下的.env文件
# load_dotenv()
# load_dotenv(override=True) 会重新读取.env
load_dotenv(override=True)

# 现在可以像访问普通环境变量一样访问.env文件中的变量了
api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('OPENAI_API_BASE')

print(api_key)
print(base_url)

chat = ChatOpenAI(
    openai_api_base=base_url,
    openai_api_key=api_key,
    model_name="moonshot-v1-8k",
    temperature=0.7,
)

sk-UGVpjuTwo2Q8pewoqUDfckw1A0pbSDli9ElFMeS9WareKknG
https://api.moonshot.cn/v1/


In [24]:
from langchain.retrievers.multi_query import MultiQueryRetriever

question = "什么是LangChain?"
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=vectordb.as_retriever(), llm=chat)

In [25]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
unique_docs

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 什么是LangChain的详细解释？', '2. LangChain的主要功能和特点是什么？', '3. 如何理解LangChain的概念和应用场景？']


[Document(page_content='我们先来看看百度百科是如何定义回调函数的：回调函数就是一个通过函数指针调用的函数。如果你把函数的指针（地址）作为参数传递给另一个函数，当这个指针被用来调用其所指向的函数时，我们就说这是回调函数。...\n\n\n\n\n\n\n\n\n参与评论\n您还未登录，请先\n登录\n后发表或查看评论\n\n\n\n\n\n\n\n\n\nLangChain入门指南\n热门推荐\n\n\n\n\nlht0909的博客\n\n\n04-27\n\n\t\t\t\t\t3万+\n\t\t\t\t\t\n\n\n\n\n\nLangChain是一个强大的框架，旨在帮助开发人员使用语言模型构建端到端的应用程序。它提供了一套工具、组件和接口，可简化创建由大型语言模型 (LLM) 和聊天模型提供支持的应用程序的过程。LangChain 可以轻松管理与语言模型的交互，将多个组件链接在一起，并集成额外的资源，例如 API 和数据库。官方文档：https://python.langchain.com/en/latest/\n\n\n\n\n\n\n\n\n\nGPT开发应用利器：LangChain\n\n\n\n\nyangbisheng1121的博客\n\n\n05-19\n\n\t\t\t\t\t5122', metadata={'description': '文章浏览阅读2.8k次。点击上方“芋道源码”，选择“设为星标”管她前浪，还是后浪？能浪的浪，才是好浪！每天 10:33更新文章，每天掉亿点点头发...源码精品专栏原创 | Java 2021超神之路，很肝~中文详细注释的开源项目RPC 框架 Dubbo 源码解析网络应用框架 Netty 源码解析消息中间件 RocketMQ 源码解析数据库中间件 Sharding-JDBC 和 MyCAT 源码解析作业调度中间件 E..._langchain', 'language': 'zh-CN', 'source': 'https://blog.csdn.net/github_38592071/article/details/131733697', 'title': '一文搞懂LangChain-CSDN博客'}),
 Document(page_content='在 LangChain 中，我们

In [32]:
from typing import List
from langchain.chains import LLMChain
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.retrievers.multi_query import LineListOutputParser

# Output parser will split the LLM result into a list of queries
# LLM 结果将被 Output 解析器拆分成查询列表。
'''
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        print(f"Input text: {text}")
        lines = text.strip().split("\n")
        return LineList(lines=lines)
'''

output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions seperated by newlines.
    Original question: {question}""",
)

# Chain
llm_chain = LLMChain(llm=chat, prompt=QUERY_PROMPT, output_parser=output_parser)

In [33]:
# Run
retriever = MultiQueryRetriever(
    retriever=vectordb.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

unique_docs = retriever.get_relevant_documents(query=question)
unique_docs

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 能解释一下LangChain是什么吗？', '2. LangChain这个术语代表什么含义？', '3. 什么是LangChain，它有哪些特点？', '4. 请描述一下LangChain的概念及其应用。', '5. LangChain是一个什么样的技术或者产品？']


[Document(page_content='在 LangChain 中，我们可以根据需要设置提示模板，并将其与主链相连接以进行输出预测。 此外，LangChain 还提供了输出解析器的功能，用于进一步精炼结果。输出解析器的作用是（1）指导模型输出的格式化方式，和（2）将输出解析为所需的格式（包括必要时的重试）。\n在 LangChain 中，我们可以提供提示模板作为输入。 模板指的是我们希望获得答案的具体格式或蓝图。LangChain 提供了预先设计好的提示模板，可以用于生成不同类型任务的提示。然而，在某些情况下，预设的模板可能无法满足你的需求。在这种情况下，我们可以使用自定义的提示模板。\n案例：\nfrom\xa0langchain\xa0import\xa0PromptTemplate', metadata={'description': '文章浏览阅读2.8k次。点击上方“芋道源码”，选择“设为星标”管她前浪，还是后浪？能浪的浪，才是好浪！每天 10:33更新文章，每天掉亿点点头发...源码精品专栏原创 | Java 2021超神之路，很肝~中文详细注释的开源项目RPC 框架 Dubbo 源码解析网络应用框架 Netty 源码解析消息中间件 RocketMQ 源码解析数据库中间件 Sharding-JDBC 和 MyCAT 源码解析作业调度中间件 E..._langchain', 'language': 'zh-CN', 'source': 'https://blog.csdn.net/github_38592071/article/details/131733697', 'title': '一文搞懂LangChain-CSDN博客'}),
 Document(page_content='我们先来看看百度百科是如何定义回调函数的：回调函数就是一个通过函数指针调用的函数。如果你把函数的指针（地址）作为参数传递给另一个函数，当这个指针被用来调用其所指向的函数时，我们就说这是回调函数。...\n\n\n\n\n\n\n\n\n参与评论\n您还未登录，请先\n登录\n后发表或查看评论\n\n\n\n\n\n\n\n\n\nLangChain入门指南\n热门推荐\n\n\n\n\nlht0909的博客\n\n\n04-27\n\n\t\t\t\t\t3万+\n\t